# Custom LSTM

https://nbviewer.jupyter.org/github/fastai/fastai/blob/master/examples/text.ipynb

## **Notes:**
- 12/02/2019 Implementation complete but results are pool better data (with actual angles and time consumed might help)

In [1]:
from devai import *
import torch.nn as nn
import pandas as pd
import torch
from torch import *
import pdb

df = pd.read_csv("dotin_data.csv")

df.head()

,Unnamed: 0,user_id,direction_list,system
0,0,365,"['SS 2.0 , SW 2.0 , SE 2.0 , SW 2.0 , SE 2.0 ,...",pc
1,1,371,"['PageChange , SE 3.0 , PageChange , NW 1.0 , ...",pc
2,2,390,"['PageChange , SW 1.0 , SE 6.0 , PageChange , ...",pc
3,3,376,"['NE 1.0 , SW 1.0 , PageChange , SW 4.0 , Page...",pc
4,4,374,"['SW 2.0 , SE 2.0 , SE 2.0 , SE 2.0 , SW 2.0 ,...",pc


In [2]:
import re
pat1 = re.compile("[.0]*[,\[\]\']*")
df["direction_list"] = df["direction_list"].apply(lambda x: re.sub(pat1,"",x))
df["direction_list"] = df["direction_list"].apply(lambda x: x.lower())

validations = pd.read_csv("validations.csv")

df = df.merge(validations,on="user_id")

df = df.sample(frac=1,random_state=2019).reset_index(drop=True)
split = round(len(df)-(.3*len(df)))
train = df.loc[:split]
val = df.loc[split:]

df.head()

,Unnamed: 0,user_id,direction_list,system,validation
0,41,464,sw 2 ss 1 se 2 se 2 sw 2 se 2 pagechange...,pc,True
1,355,1375,se 3 sw 2 sw 2 ss 2 nw 2 se 2 se 2 page...,pc,True
2,307,1324,sw 1 se 2 sw 2 se 2 sw 2 nn 2 se 2 page...,pc,True
3,530,1686,sw 2 nw 2 ss 2 se 3 sw 2 se 2 sw 2 sw 2...,pc,False
4,547,1710,se 3 sw 1 ss 2 sw 3 se 2 sw 2 ne 3 se 2...,pc,True


In [3]:
df["direction_list"][0].split("  ")[:5]

['sw 2', 'ss 1', 'se 2', 'se 2', 'sw 2']

In [4]:
bos = [0,0]
eos = [500,15]
pad = [370,11]

In [5]:
def to_tensor(idx):
    dct = {"ee":0,"se":45,"ss":90,"sw":135,"ww":180,"nw":225,"nn":270,"ne":315}
    x = [bos]
    for i in df["direction_list"][idx].split("  "):
        splt = i.split()
        if len(splt) == 2 and len(splt[0])==2:
            res = list(map(dct.get,[splt[0]]))
            res.append(int(splt[1]))
            x.append(res)
        else:
            x.append(pad)
    return tensor(x+[eos])

In [6]:
class Movement:
    def __init__(self,id,tensor): self.id,self.tensor = id,tensor 
    def get(self): return self.tensor
    def __repr__(self): return str(self.tensor)
    def __len__(self): return len(self.tensor)
    def __getattr__(self,k): return getattr(self.tensor,k)
    def __getitem__(self, idx): return self.tensor.item()

In [7]:
moves = [Movement(df["user_id"][i],to_tensor(i)) for i in range(len(df))]
il = ItemList(moves)
sd = SplitData.split_by_func(il,partial(random_splitter,p_valid=.3))

labels = {df["user_id"][i]:df["validation"][i] for i in range(len(df))}
def id_labeller(mm): return labels[mm.id]

ll = label_by_func(sd,id_labeller)

In [8]:
class LM_PreLoader():
    def __init__(self, data, bs=64, bptt=70, shuffle=False):
        self.data,self.bs,self.bptt,self.shuffle = data,bs,bptt,shuffle
        total_len = sum([len(t) for t in data.x])
        self.n_batch = total_len // bs
        self.batchify()
    
    def __len__(self): return ((self.n_batch-1) // self.bptt) * self.bs
    
    def __getitem__(self, idx):
        source = self.batched_data[idx % self.bs]
        seq_idx = (idx // self.bs) * self.bptt
        return source[seq_idx:seq_idx+self.bptt],source[seq_idx+1:seq_idx+self.bptt+1]
    
    def batchify(self):
        tensors = self.data.x
        if self.shuffle: tensors = tensors[torch.randperm(len(tensors))]
        stream = torch.cat([t.tensor for t in tensors])
        self.batched_data = stream[:self.n_batch * self.bs].view(self.bs, self.n_batch,2) # the 2 is for the two values per movement
        

In [9]:
dl = DataLoader(LM_PreLoader(ll.train, shuffle=True), batch_size=64)

iter_dl = iter(dl)
x1,y1 = next(iter_dl)
x2,y2 = next(iter_dl)

x1.size(),y1.size()

(torch.Size([64, 70, 2]), torch.Size([64, 70, 2]))

In [10]:
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
def get_lm_dls(train_ds, valid_ds, bs, bptt, **kwargs):
    return (DataLoader(LM_PreLoader(train_ds, bs, bptt, shuffle=True), batch_size=bs, **kwargs),
            DataLoader(LM_PreLoader(valid_ds, bs, bptt, shuffle=False), batch_size=bs, **kwargs))


def lm_databunchify(sd, bs, bptt, **kwargs):
    return DataBunch(*get_lm_dls(sd.train, sd.valid, bs, bptt, **kwargs))

bs,bptt = 64,70
data = lm_databunchify(ll, bs, bptt)

# LM Model

In [11]:
# class LSTM(nn.Module):
#     def __init__(self):
#         self.lstm1 = nn.LSTM(2, 50)  # Input dim is 3, output dim is 3
#         self.lstm2 = nn.LSTM(50,2)

# # initialize the hidden state.
# hidden = (torch.randn(1, 1, 3),
#           torch.randn(1, 1, 3))
# for i in inputs:
#     # Step through the sequence one element at a time.
#     # after each step, hidden contains the hidden state.
#     out, hidden = lstm(i.view(1, 1, -1), hidden)

In [12]:
class BatchNorm1dFlat(nn.BatchNorm1d):
    "`nn.BatchNorm1d`, but first flattens leading dimensions"
    def forward(self, x):
        if x.dim()==2: return super().forward(x)
        *f,l = x.shape
        x = x.contiguous().view(-1,l)
        return super().forward(x).view(*f,l)

In [13]:
wq = torch.rand(1,64,50); wq[:,13:].shape

torch.Size([1, 51, 50])

In [14]:
torch.cat((wq[:,13:],torch.rand(1,13,50)),dim=1).shape

torch.Size([1, 64, 50])

In [15]:
class GRU(nn.Module):
    def __init__(self,nv,nh,nl,bs):
        super().__init__()
        self.bs = bs
        self.rnn = nn.GRU(nv, nh, nl, batch_first=True)
        self.h = torch.zeros(nl, bs, nh).cuda()
        
    def forward(self, x):
        bs,sl,*_ = x.shape
        
        # working with shorter batch
        if bs < self.bs: 
            res, h = self.rnn(x,self.h[:,-bs:].contiguous())
            self.h = torch.cat((self.h[:,bs:],h.detach()),dim=1)
        else:
            res,h = self.rnn(x, self.h)
            self.h = h.detach()
        return res,h
    

class LinearDecoder(nn.Module):
    def __init__(self, nv, nh ,lm=True):
        super().__init__()
        self.h_o = nn.Linear(nh,nv)
        self.lm = lm

    def forward(self, x):
        return self.h_o(x[0]) if self.lm else self.h_o(x[1])[0]

In [16]:
m,s = x1.float().mean((0,1)), x1.float().std((0,1)); m,s

def normalize_feat(x, mean, std):
    return (x.float()-mean)/std

class BatchTransformYCallback(Callback):
    _order=2
    def __init__(self, tfm): self.tfm = tfm
    def after_pred(self): self.run.yb = self.tfm(self.yb)

def MSE(outp,targ): return (outp-targ).pow(2)
def MSE_combined(outp,targ): return MSE(outp,targ).mean((0,1)).sum()
def MSE_scaled_dir(outp,targ,feat_idx=0,mean=m.cuda(),std=s.cuda()): 
    return (MSE(outp,targ).mean((0,1))*std + mean)[0]
def MSE_scaled_mag(outp,targ,feat_idx=0,mean=m.cuda(),std=s.cuda()): 
    return (MSE(outp,targ).mean((0,1))*std + mean)[1]

In [17]:
norm_dotin = partial(normalize_feat,mean=m.cuda(),std=s.cuda())
cbfs = [partial(AvgStatsCallback,[MSE_scaled_dir,MSE_scaled_mag]),
        CudaCallback,
        partial(BatchTransformXCallback, norm_dotin),
        partial(BatchTransformYCallback, norm_dotin),
       ProgressCallback]

lm_model = nn.Sequential(GRU(2,50,3,bs),LinearDecoder(2,50),BatchNorm1dFlat(2))

learn = Learner(lm_model,data,MSE_combined,cb_funcs=cbfs,lr=.02)

In [18]:
learn.fit(10)

epoch,train_loss,train_MSE_scaled_dir,train_MSE_scaled_mag,valid_loss,valid_MSE_scaled_dir,valid_MSE_scaled_mag,time
0,2.594610,293.552979,6.910175,1.977680,254.229248,6.188035,00:00
1,1.940071,256.651825,6.018651,1.960636,253.791534,6.151214,00:00
2,1.814799,249.777649,5.843126,1.923908,252.672729,6.076431,00:00
3,1.783854,247.895615,5.804549,1.866994,250.889221,5.961843,00:00
4,1.783097,247.842773,5.803783,1.857516,249.760727,5.964387,00:00
5,1.777419,247.520126,5.796115,1.883115,250.540695,6.016505,00:00
6,1.774448,247.361389,5.791842,1.884763,250.765076,6.015330,00:00
7,1.772298,247.251633,5.788613,1.876366,250.526611,5.997781,00:00
8,1.770467,247.161484,5.785780,1.866953,250.218170,5.979181,00:00
9,1.768763,247.079620,5.783088,1.857525,249.891235,5.961017,00:00


In [19]:
# saving the model
st = learn.model.state_dict()
torch.save(st, "dotin_lm")

# Loading Classification Data

In [20]:
moves = [Movement(df["user_id"][i],to_tensor(i)) for i in range(len(df))]
il = ItemList(moves)
sd = SplitData.split_by_func(il,partial(random_splitter,p_valid=.3))

labels = {df["user_id"][i]:df["validation"][i] for i in range(len(df))}
def id_labeller(mm): return labels[mm.id]

proc_cat = CategoryProcessor()
ll = label_by_func(sd,id_labeller,proc_y=proc_cat)

In [21]:
from torch.utils.data import Sampler

class SortSampler(Sampler):
    def __init__(self, data_source, key): self.data_source,self.key = data_source,key
    def __len__(self): return len(self.data_source)
    def __iter__(self):
        return iter(sorted(list(range(len(self.data_source))), key=self.key, reverse=True))

In [22]:
class SortishSampler(Sampler):
    def __init__(self, data_source, key, bs):
        self.data_source,self.key,self.bs = data_source,key,bs

    def __len__(self) -> int: return len(self.data_source)

    def __iter__(self):
        idxs = torch.randperm(len(self.data_source))
        megabatches = [idxs[i:i+self.bs*50] for i in range(0, len(idxs), self.bs*50)]
        sorted_idx = torch.cat([tensor(sorted(s, key=self.key, reverse=True)) for s in megabatches])
        batches = [sorted_idx[i:i+self.bs] for i in range(0, len(sorted_idx), self.bs)]
        max_idx = torch.argmax(tensor([self.key(ck[0]) for ck in batches]))  # find the chunk with the largest key,
        batches[0],batches[max_idx] = batches[max_idx],batches[0]            # then make sure it goes first.
        batch_idxs = torch.randperm(len(batches)-2)
        sorted_idx = torch.cat([batches[i+1] for i in batch_idxs]) if len(batches) > 1 else LongTensor([])
        sorted_idx = torch.cat([batches[0], sorted_idx, batches[-1]])
        return iter(sorted_idx)

In [23]:
def pad_collate_emb(samples, pad_emb=pad, pad_first=False):
    max_len = max([len(s[0]) for s in samples])
    res = torch.zeros(len(samples), max_len, 2) + torch.tensor(pad_emb).float() 
    for i,s in enumerate(samples):
        if pad_first: res[i, -len(s[0]):] = s[0].tensor
        else:         res[i, :len(s[0])] = s[0].tensor
    return res, tensor([s[1] for s in samples])

In [24]:
bs = 64
train_sampler = SortishSampler(ll.train.x, key=lambda t: len(ll.train[int(t)][0]),bs=bs)
train_dl = DataLoader(ll.train, batch_size=bs, sampler=train_sampler, collate_fn=pad_collate_emb)

valid_sampler = SortSampler(ll.valid.x,key=lambda t: len(ll.train[int(t)][0]))
valid_dl = DataLoader(ll.valid,batch_size=bs,sampler=valid_sampler, collate_fn=pad_collate_emb)

In [25]:
iter_dl = iter(train_dl)
x,y = next(iter_dl)
x2,y2 = next(iter_dl)

In [26]:
x.shape,x2.shape # notice these are different

(torch.Size([64, 277, 2]), torch.Size([64, 159, 2]))

In [27]:
data = DataBunch(train_dl,valid_dl,c_out=2)

# Classification Model

In [28]:
norm_dotin = partial(normalize_feat,mean=m.cuda(),std=s.cuda())
cbfs = [partial(AvgStatsCallback,accuracy),
        CudaCallback,
        partial(BatchTransformXCallback, norm_dotin),
       ProgressCallback]

In [29]:
# loading the previous model
learn = Learner(lm_model, data, loss_func=F.cross_entropy,cb_funcs=cbfs,lr=.001)

In [30]:
st = torch.load("dotin_lm")
learn.model.load_state_dict(st)

<All keys matched successfully>

In [31]:
# cutting off old model
learn.model = learn.model[:1] # 1 is the cut off since we only want to export the GRU

In [32]:
learn.model.add_module("1",LinearDecoder(2,50,lm=False))
learn.model.add_module("2",BatchNorm1dFlat(2))

In [33]:
for p in learn.model[0].parameters(): p.requires_grad_(False) # model[0] is m_cut (our base model from previous example)
    

In [34]:
len(data.train_ds) % 64

6

In [35]:
learn.fit(100)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,0.990416,0.405286,0.711054,0.289216,00:00
1,0.976260,0.409692,0.700201,0.289216,00:00
2,0.923600,0.396476,0.678752,0.735294,00:00
3,0.795008,0.403084,0.673587,0.735294,00:00
4,0.765222,0.398678,0.676990,0.735294,00:00
5,0.744197,0.411894,0.685798,0.735294,00:00
6,0.739224,0.403084,0.698524,0.299020,00:00
7,0.729831,0.416300,0.704079,0.289216,00:00
8,0.717617,0.416300,0.713675,0.289216,00:00
9,0.723138,0.403084,0.719091,0.279412,00:00


In [37]:
1-torch.tensor(ll.valid.y.items).float().mean()

tensor(0.7353)

In [38]:
1-torch.tensor(ll.train.y.items).float().mean()

tensor(0.7070)